In [ ]:
import sys
import os
from glob import glob

import numpy as np
import matplotlib.pyplot as plt

import pydicom

from pymedphys.gamma import gamma_shell, GammaInternalFixedOptions, calculate_min_dose_difference, convert_to_percent_pass

In [ ]:
x = 1

x = np.array(x)[None]
len(np.shape(x))

In [ ]:
DATA_DIRECTORY = "../../../tests/data/gamma/agnew_mcgarry_images"
glob(DATA_DIRECTORY)

In [ ]:
REF_VMAT_1mm = os.path.abspath(os.path.join(
    DATA_DIRECTORY, 'H&N_VMAT_Reference_1mmPx.dcm'))
EVAL_VMAT_1mm = os.path.abspath(os.path.join(
    DATA_DIRECTORY, 'H&N_VMAT_Evaluated_1mmPx.dcm'))

In [ ]:
def load_dose_from_dicom(dcm):
    pixels = dcm.pixel_array
    dose = pixels * dcm.DoseGridScaling

    return dose


def load_yx_from_dicom(dcm):
    resolution = np.array(
        dcm.PixelSpacing).astype(float)
    dx = resolution[0]
    x = (
        dcm.ImagePositionPatient[0] +
        np.arange(0, dcm.Columns * dx, dx))

    dy = resolution[1]
    y = (
        dcm.ImagePositionPatient[1] +
        np.arange(0, dcm.Rows * dy, dy))

    return y, x

In [ ]:
dcm_ref = pydicom.read_file(REF_VMAT_1mm)
dcm_eval = pydicom.read_file(EVAL_VMAT_1mm)

coords_reference = load_yx_from_dicom(dcm_ref)
dose_reference = load_dose_from_dicom(dcm_ref)

coords_evaluation = load_yx_from_dicom(dcm_eval)
dose_evaluation = load_dose_from_dicom(dcm_eval)

In [ ]:
gamma = gamma_shell(
    coords_reference, dose_reference,
    coords_evaluation, dose_evaluation,
    [1], [1], max_gamma=1.0001, skip_once_passed=True)

In [ ]:
# import itertools

# itertools.product(np.array(1),np.array(2))

In [ ]:
def get_dummy_gamma_set():
    grid_x = np.arange(0, 1, 0.1)
    grid_y = np.arange(0, 1.2, 0.1)
    grid_z = np.arange(0, 1.4, 0.1)
    dimensions = (len(grid_x), len(grid_y), len(grid_z))
    coords = (grid_x, grid_y, grid_z)

    reference = np.zeros(dimensions)
    reference[3:-2:, 4:-2:, 5:-2:] = 1.015

    evaluation = np.zeros(dimensions)
    evaluation[2:-2:, 2:-2:, 2:-2:] = 1

    expected_gamma = np.zeros(dimensions)
    expected_gamma[2:-2:, 2:-2:, 2:-2:] = 0.4
    expected_gamma[3:-3:, 3:-3:, 3:-3:] = 0.7
    expected_gamma[4:-4:, 4:-4:, 4:-4:] = 1
    expected_gamma[3:-2:, 4:-2:, 5:-2:] = 0.5

    return coords, reference, evaluation, expected_gamma


coords, reference, evaluation, _ = get_dummy_gamma_set()

In [ ]:
coords, reference, evaluation, _ = get_dummy_gamma_set()

gamma = gamma_shell(
    coords, reference,
    coords, evaluation,
    [3, 5], [0.3, 0.5], lower_percent_dose_cutoff=0)

In [ ]:
def calc_gamma_for_case(distance, threshold_ratios):
    options = GammaInternalFixedOptions.from_user_inputs(
        coords, reference,
        coords, evaluation,
        distance * threshold_ratio, distance, lower_percent_dose_cutoff=0, quiet=True
    )

    still_searching_for_gamma = np.ones_like(
        options.flat_dose_reference).astype(bool)
    current_gamma = np.inf * np.ones_like(options.flat_dose_reference)
    distance = 0.0

    while distance <= options.maximum_test_distance:
        to_be_checked = (
            options.reference_points_to_calc & still_searching_for_gamma)

        min_relative_dose_difference = calculate_min_dose_difference(
            options, distance, to_be_checked)
        
        for 

        gamma_at_distance = np.sqrt(
            (min_relative_dose_difference /
             (options.dose_percent_threshold / 100)) ** 2
            + (distance / options.distance_mm_threshold) ** 2)

        current_gamma[to_be_checked] = np.min(
            np.vstack((
                gamma_at_distance, current_gamma[to_be_checked]
            )), axis=0)

        still_searching_for_gamma = (
            current_gamma > distance / options.distance_mm_threshold)

        if options.skip_once_passed:
            still_searching_for_gamma = (
                still_searching_for_gamma & (current_gamma >= 1))

        distance += options.distance_step_size

        if np.sum(to_be_checked) == 0:
            break

    gamma = np.reshape(
        current_gamma, np.shape(reference))
    gamma[np.isinf(gamma)] = np.nan
    
    return gamma

In [ ]:
base_dist = 0.3
threshold_ratios = np.arange(1,7)

gamma_store = {}
for ratio in threshold_ratios:
    gamma = calc_gamma_for_case(base_dist, ratio)
    gamma_store[ratio] = gamma



# convert_to_percent_pass


In [ ]:
# convert_to_percent_pass(gamma_store[5])

In [ ]:
gamma_scaling = np.arange(1, 5, 0.1)

distance_thresholds = []
dose_thresholds = []
pass_rates = []

for scale in gamma_scaling:
    for threshold_ratio, gamma in gamma_store.items():
        pass_rates.append(convert_to_percent_pass(gamma_store[threshold_ratio] / scale))
        distance_thresholds.append(base_dist * scale)
        dose_thresholds.append(distance_thresholds[-1] * threshold_ratio)
        

plt.scatter(distance_thresholds, dose_thresholds, c=pass_rates, vmin=90, vmax=100, cmap='coolwarm_r')
plt.colorbar()
plt.xlim([0.3, 1.5])
plt.ylim([0.2, 3])

plt.xlabel('Distance Threshold')
plt.ylabel('Dose Threshold')



In [ ]:
# pass_rates

In [ ]:
dose_thresholds

In [ ]:
plt.contourf(threshold_ratios, )

In [ ]:
plt.pcolormesh(gamma[5,:,:])
plt.colorbar()